# Projeto de Reconhecimento de Emoções com CNN

## Cronograma

### Semana 24/10 a 31/10
- Carregar e organizar um dataset de imagens próprias (mínimo 3 classes)
- Aplicar técnicas de padronização:
  - Redimensionamento
  - Grayscale
  - Normalização
- Implementar e treinar uma CNN simples (SimpleCNN - PyTorch)

### Semana 31/10 a 07/11
- Aplicar validação cruzada (k-fold cross-validation)
  - *Justificativa: Necessário devido ao tamanho reduzido dos datasets*
- Avaliar o desempenho do modelo com métricas
- Discussão sobre os resultados

**Data de Entrega:** 03 a 07 de Novembro

Dataset:

O dataset utilizado neste projeto foi obtido de [https://drive.google.com/drive/u/0/folders/1_4L4nw7XvlGiV3guMkyPaPVQgNC9_zmR].

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import cv2

In [ ]:
train_dataset_path = './data/train'
test_dataset_path = './data/test'

classes = os.listdir(train_dataset_path)

classes = [dir for dir in classes if os.path.isdir(os.path.join(train_dataset_path, dir))]

print(f"\nClasses: {len(classes)}")
for classe in classes:
    print(classe)

In [ ]:
classes = os.listdir(train_dataset_path)
print("Classes found:", classes)

plt.figure(figsize=(15, 5))
for i, emotion in enumerate(classes):
    emotion_path = os.path.join(train_dataset_path, emotion)
    random_img = random.choice(os.listdir(emotion_path))
    img_path = os.path.join(emotion_path, random_img)

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(1, len(classes), i+1)
    plt.imshow(img)
    plt.title(emotion)
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
IMG_SIZE = 64
EPOCHS = 50
BATCH_SIZE = 32
NUM_CLASSES = 6

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

In [ ]:
val_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
)

In [ ]:
classes = list(train_generator.class_indices.keys())
aumento = 5  

plt.figure(figsize=(15, len(classes) * 3))

for c, class_name in enumerate(classes):
    class_index = train_generator.class_indices[class_name]
    images, labels = next(train_generator)
    idx = np.where(labels[:, class_index] == 1)[0][0]
    img = images[idx]

    plt.subplot(len(classes), aumento + 1, c * (aumento + 1) + 1)
    plt.imshow(img.reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
    plt.title(f"{class_name}\nOriginal")
    plt.axis('off')

    for i in range(aumento):
        aug_img = train_datagen.random_transform(img)
        plt.subplot(len(classes), aumento + 1, c * (aumento + 1) + i + 2)
        plt.imshow(aug_img.reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
        plt.title(f"Aumento {i+1}")
        plt.axis('off')

plt.suptitle("Original e versões aumentadas", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Modelo sequencial keras com entrada de ima imagem 64x64 um canal e saida em vetor com uma probabilidade por classe
model = Sequential([
    # 64 filtros convolucionais, cada um 3×3 → detectam bordas, contornos e padrões simples.
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 1)),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # 128 filtros, captando padrões mais complexos.
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # 256 filtros, capturarando combinações de formas.
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # 512 filtros, reconhecendo caracteristicas muito especificas e compostas
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Achatamento, normalização e redução de overfitting com saida entre 0 e 1 por classe
    Flatten(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
# Prepara o modelo para treino e detalha a arquitetura da rede
# define aprendizado e avaliação
# loss mede o erro entre previsões e rótulos verdadeiros
# define a métrica de avaliação
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# interrompe o treinamento automaticamente quando o modelo para de melhorar
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
# reduz automaticamente a taxa de aprendizado (learning_rate) quando o modelo para de melhorar.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [ ]:
# Tratamento para datasets com classes desbalanceadas
class_labels = train_generator.classes
num_classes = train_generator.num_classes
# Classes com menos amostras = maior peso | classes com mais amostras = menor peso
class_weights_array = class_weight.compute_class_weight(
    "balanced",
    classes = np.unique(class_labels),
    y = class_labels
)

class_weights = dict(enumerate(class_weights_array))
# todos terão peso 1.0 nesse dataset pois ele contém
# o mesmo número de imagens para cada classe
print("Calculated Class Weights:", class_weights)

In [ ]:
# Treinamento
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stop, reduce_lr],
    class_weight = class_weights,
)

In [ ]:
# Mede a acurácia
test_loss, test_acc = model.evaluate(test_generator)
print(f"Acurácia: {test_acc:.4f}")

In [ ]:
# Avaliação do modelo
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

print(classification_report(y_true, y_pred_classes, target_names=train_generator.class_indices.keys()))

In [ ]:
# matriz de confusão
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=train_generator.class_indices.keys(), yticklabels=train_generator.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Gráfico de desempenho ao longo das épocas
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()